<div class='alert alert-info'>
    <h1>MarketingCloud Salesforce SFTP</h1>
</div>

In [1]:
import cx_Oracle
import pysftp
import pandas as pd
import os
import csv
import time
import datetime
import paramiko
import base64
import paramiko
from scp import SCPClient
from datetime import datetime

In [2]:
class config_bd:
    username = 'user_name'
    password = 'password$'
    host = 'host' 
    port = 2398041208973478091278394
    dbname = 'dbname'
    sdi = 'sdi'
    encoding = 'UTF-8'

In [3]:
def bd_connect():
    """Doc String"""
    dns = cx_Oracle.makedsn(config_bd.host, config_bd.port, config_bd.sdi)
    print(dns)
    while True:
        try:
            con = cx_Oracle.Connection(config_bd.username,config_bd.password,dns)
            now = datetime.now()
            dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
            print(f"Conexión exitosa, versión: {con.version}, fecha: {dt_string}")
            break
        except:
            now = datetime.now()
            dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
            print(f"No se puede conectar a la base de datos, fecha: {dt_string}")
            #time.sleep(60)
    return con

def bd_query(query,con):
    """Doc String"""
    df = pd.read_sql(sql=query,con=con)
    return df

def save_file(df,name):
    #now = datetime.now()
    #dt_string = now.strftime("%d_%m_%Y-%H_%M_%S")
    #name = "TriggreredFile_"+dt_string+".csv"
    df.to_csv(name,quoting=csv.QUOTE_ALL,index=False,encoding="utf-8-sig")
    return name

def bd_close(con):
    """Doc String"""
    con.close()
    return None

def ftp_connect():
    """Doc String"""
    try:
        ssh = paramiko.SSHClient()
        ssh.load_system_host_keys()
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh.connect('ftp.s7.exacttarget.com', username='username', password='password',timeout=10)
        hstk = ssh.get_host_keys()
        cnopts = pysftp.CnOpts()
        cnopts.hostkeys = hstk
        sftp = pysftp.Connection('ftp.s7.exacttarget.com',username='username',password='password',port=22,cnopts=cnopts)
        print("Connected to sftp")
        return sftp,ssh
    except:
        print("Error conectandoce al servidor")
        return False, False

def ftp_send(sftp,file_name):
    """Doc String"""
    try:
        sftp.remove('/Triggered_Automations/'+file_name) #Eliminamos el archivo viejo
        sftp.put(file_name,'/Triggered_Automations/'+file_name) #Ponemos el nuevo archivo
        #ftp_ssh_close(sftp,ssh)
        print("Archivo enviado, bloque try")
    except: 
        sftp.put(file_name,'/Triggered_Automations/'+file_name)  # upload file to public/ on remote
        #ftp_ssh_close(sftp,ssh)
        print("Archivo enviado, bloque except")

def ftp_ssh_close(sftp,ssh):
    sftp.close()
    ssh.close()
    print("Connection clossed")

def main(con,df_r):
    """Doc String"""
    query = """query"""
    
    df_query = bd_query(query,con)
    df_to_save = df_r.append(df_query)
    df_to_save.drop_duplicates(subset='SUBSCRIBERKEY',keep='last',inplace=True)
    name = save_file(df_to_save,'202006_VIAJACERTIFICADOS.csv')
    sftp,ssh = ftp_connect()
    if sftp != False and ssh != False:
        ftp_send(sftp,name)
        return sftp,ssh
    else:
        print("Hubo un problema con el servidor SFTP")
        return False,False

In [ ]:
if __name__ == "__main__":
    while True:
        if os.path.isfile('./202006_VIAJACERTIFICADOS.csv'):
            df = pd.read_csv('202006_VIAJACERTIFICADOS.csv')
            con = bd_connect()
            sftp,ssh = main(con,df)
            if sftp != False and ssh != False:
                bd_close(con)
                ftp_ssh_close(sftp,ssh)
                time.sleep(600)
            else:
                print("Volvemos a intentar el proceso")
                continue
        else: 
            df_skeleton = pd.DataFrame(
                {
                    'SUBSCRIBERKEY':[],
                    'MEMBER_NUMBER':[],
                    'CLIENT_TYPE':[],
                    'NAME':[],
                    'LAST_NAME':[],
                    'EMAIL':[],
                    'PHONE':[],
                    'PROMOTION_ID':[],
                    'HOTEL_CODE':[],
                    'RATE_TYPE':[], 
                    'TIME_STAMP':[],
                    'STATUS_CODE':[]
                }
            )
            save_file(df_skeleton,name='202006_VIAJACERTIFICADOS.csv')